In [28]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

True

In [29]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
def make_completion(prompt: str) -> str:
    answer = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": prompt,
            },
        ]
    )
    return answer.choices[0].message.content

In [5]:
PROMPT_TEMPLATE = """I will give you the name of the book, your task is to search for it in the internet and construct one sentence what the book is about. I will need this sentence in order to determine the topics of this book.
If you can not find the information about the book you should tell it.

Book name:
{book_name}

Return the answer in the following JSON format:
{{
  "know": <str, 'yes' or 'no'>,
  "sentence": <str, the sentence that tells what the book is about. Do not return this key if 'know' keys is no>
}}
Do not return additional text, except json answer."""

In [6]:
import re
import json

def get_summary(book_name: str):
    prompt = PROMPT_TEMPLATE.format(book_name=book_name)
    response = make_completion(prompt)
    if not response:
        return Exception("No response")
    match = re.search(r"\{.*\}", response, re.DOTALL)
    if match:
        data = json.loads(match.group(0))
        if "sentence" in data:
            return data["sentence"]
        else:
            return Exception("No sentence key in json: {}".format(response))
    else:
        return Exception("Wrong json output: {}".format(response))

In [27]:
get_summary("The Infinite Game by Simon Sinek")

'The Infinite Game by Simon Sinek explores the concept of business and leadership as an infinite game, focusing on vision and societal impact rather than finite goals and immediate success.'

In [21]:
import pandas as pd

df = pd.read_csv("save/data.csv")

In [9]:
from tqdm.auto import tqdm

summaries = {}
for book_name in tqdm(df["name"]):
    summary = get_summary(book_name)
    if isinstance(summary, str):
        summaries[book_name] = summary
    else:
        print(book_name)
        summaries[book_name] = book_name

100%|██████████| 1216/1216 [37:36<00:00,  1.86s/it]


In [10]:
json.dump(summaries, open("save/summaries.json", "w"))

In [26]:
get_summary("StrengthsFinder 2.0 by Tom Rath")

'StrengthsFinder 2.0 by Tom Rath is a book that helps readers identify their top strengths and offers strategies to utilize them effectively for personal and professional growth.'

In [ ]:
# Books that don't have summaries

"The Richest Man in Babylon — Six Laws of Wealth by Charles Conrad"
"Me: Elton John Official Biography by Elton John"
"The Coronavirus Pandemic: An essential guide to the SARS-CoV-2, COVID-19, and the key measures for safety by Headway Editorial Team"
"Self-Discipline in Difficult Times: Pressing Ahead (or Not) When Your World Turns Upside Down by Martin Meadows"
"The Law of Attraction: How to Attract Money, Love, and Happiness by David R. Hooper"
"Million Dollar Habits: 27 Powerful Habits to Wire Your Mind for Success, Become Truly Happy, and Achieve Financial Freedom by Stellan Moreira"
"The Magic of Believing: Unlock Your Potentials by Claude. M. Bristol"
"Negotiation Hacks: Expert Tactics To Get What You Want by Simon Rycraft"
"Have a New Kid by Friday: How To Change Your Child’s Attitude, Behavior and Character in 5 Days by Kevin Lemar, Ph.D."
"Exceptional: Build Your Personal Highlight Reel and Unlock Your Potential by Daniel M. Cable"
"The Side Hustle: A Beginners Guide to Level-Up from a 9-5 to 6 Side Hustles That Will Create Multiple Streams of Income by Aubrey Hall"
"The book to test and delete by Mark Hyman, M.D."
"How to Talk Anyone: 92 Little Tricks for Big Success in Relationships - test by Leil Lowndes"
"The 5 Love Languages: The Secret to Love that Lasts - test by Gary D. Chapman, Ph.D."
"13 Things Mentally Strong People Don’t Do: Take Back Your Power, Embrace Change, Face Your Fears, and Train Your Brain for Happiness and Success - test by Amy Morin"
"Have a New Sex Life by Friday: Because Your Marriage Can’t Wait Till Monday - test by Dr. Kevin Leman"
"Music Theory: From Absolute Beginner to Expert — The Ultimate Step-By-Step Guide to Understanding and Learning Music Theory Effortlessly - test by Nicolas Carter"
"CHATGPT BOOK FOR BEGINNERS: Getting Started with ChatGPT, The Ultimate Beginner's Guide to Use ChatGPT Effectively, Earn Money and Increase Your Productivity 10x by Arsath Natheem"